In [29]:
!pip install requests

In [30]:
import requests
import json
from IPython.display import Image
import os

In [31]:
api_key='rRtAMsJ6LJjcXv4z5JKCs1Q7IeQ3tdXTb3bFrBw8'
url=f'https://api.nasa.gov/planetary/apod?api_key={api_key}'
response = requests.get(url)
data = response.json()
data

{'copyright': 'Alexsandro Mota',
 'date': '2025-07-12',
 'explanation': "As the Sun set, a bright Full Moon rose on July 10. Its golden light illuminates clouds drifting through southern hemisphere skies in this well-composed telephoto image from Conceição do Coité, Bahia, Brazil. The brightest lunar phase is captured here with both a short and long exposure. The two exposures were combined to reveal details of the lunar surface in bright moonlight and a subtle iridescence along the dramatically backlit cloudscape. Of course, July's Full Moon is a winter moon in the southern hemisphere.  But in the north it's known to some as the Thunder Moon, likely a nod to the sounds of this northern summer month's typically stormy weather.",
 'hdurl': 'https://apod.nasa.gov/apod/image/2507/LUA_JULHO_25_2048.jpg',
 'media_type': 'image',
 'service_version': 'v1',
 'title': 'Clouds and the Golden Moon',
 'url': 'https://apod.nasa.gov/apod/image/2507/LUA_JULHO_25_1024.jpg'}

In [32]:
image_url = data['hdurl']
# image_url
Image(url=image_url,width=500)

In [33]:
apod_date = data['date']
apod_date

'2025-07-12'

In [34]:
file_extension = os.path.splitext(image_url)[1]
filename = f"apod_{apod_date}{file_extension}" # Image will be saved in the current directory

# Download and save the image
image_response = requests.get(image_url, stream=True)
with open(filename, 'wb') as f: # Saves directly to the current directory
    for chunk in image_response.iter_content(1024):
        f.write(chunk)

print(f"Image saved to: {os.getcwd()}/{filename}")

Image saved to: c:\Users\kkapil\Documents\mlops-python\16-astro\dags/apod_2025-07-12.jpg


#### Connect to postgres db

In [35]:
%pip install psycopg2-binary pandas tabulate


In [36]:
# Cell 2: Import necessary libraries
import psycopg2
import pandas as pd # For better data handling and display in Jupyter

In [37]:
# Cell 3: Database Connection Parameters
# Replace these with your actual remote PostgreSQL database details
db_host = "database-1.cylcqsak4ngs.us-east-1.rds.amazonaws.com"  # e.g., database-1.cylccqsa.us-east-1.rds.amazonaws.com
db_name = "postgres"
db_user = "postgres"
db_password = "jIBZbI0vDxphkYSJwa7Q"
db_port = "5432"  # Default PostgreSQL port

In [38]:
# Cell 4: Establish Connection and Retrieve Table Names
# Establish the connection
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

# Create a cursor object
cur = conn.cursor()
sql_query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_type = 'BASE TABLE';
"""

# Execute the query
cur.execute(sql_query)

# Fetch all table names
table_names = [row[0] for row in cur.fetchall()]

print("Tables in the database:")
for table in table_names:
    print(f"- {table}")

# Close the cursor and connection
cur.close()
conn.close()
print("\nPostgreSQL connection closed.")

Tables in the database:
- apod_data

PostgreSQL connection closed.


In [39]:
table_to_fetch = "apod_data"
# Set display options to show full content for wide columns
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', None) # Or a large number like 1000

# print(df) # Print your DataFrame again

conn = None # Initialize conn to None for safety
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password,
        port=db_port
    )

    # Create a cursor object
    cur = conn.cursor()

    # SQL query to select all data from the chosen table
    # Using f-strings to embed the table name. Sanitize inputs in production!
    sql_query = f"SELECT * FROM {table_to_fetch};"

    # Execute the query
    cur.execute(sql_query)

    # Fetch all rows
    rows = cur.fetchall()

    # Get column names from the cursor description
    column_names = [desc[0] for desc in cur.description]

    # Convert to a Pandas DataFrame for easy viewing
    df = pd.DataFrame(rows, columns=column_names)

    print(f"Data fetched successfully from table '{table_to_fetch}':")
    print(df.head())
    # print(df.to_string(index=False))
    # print(df[['title', 'date', 'media_type']])

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL or fetching data: {e}")
finally:
    # Close the cursor and connection
    if conn: # Ensure conn exists before trying to close
        cur.close()
        conn.close()
        print("\nPostgreSQL connection closed.")

Data fetched successfully from table 'apod_data':
   id                       title  \
0   1         The Veins of Heaven   
1   2  Clouds and the Golden Moon   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   explanation  \
0  Transfusing sunlight as the sky grew darker, this exceptional display of noctilucent clouds was c

In [40]:
from tabulate import tabulate

# Set options to display full DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Assuming df is your DataFrame with multiple rows
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))
print(f"The DataFrame has {len(df)} rows.")


+------+----------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------+------------+--------------+
|   id | title                      | explanation                                                                                       